In [ ]:
# Cuda config
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}\nLogical: {len(tf.config.experimental.list_logical_devices('GPU'))}")

Num GPUs Available: 1
Logical: 1


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
import random

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_paths, mask_paths, labels, batch_size=32, is_segmentation=True, input_size=(256, 256), num_classes=4, shuffle=True):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.labels = labels
        self.batch_size = batch_size
        self.is_segmentation = is_segmentation
        self.input_size = input_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.image_paths) // self.batch_size

    def __getitem__(self, index):
        batch_image_paths = self.image_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_mask_paths = self.mask_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        
        images, masks, labels = self.__data_generation(batch_image_paths, batch_mask_paths, batch_labels)
        return [images, masks], labels

    def on_epoch_end(self):
        if self.shuffle:
            indices = np.arange(len(self.image_paths))
            np.random.shuffle(indices)
            self.image_paths = [self.image_paths[i] for i in indices]
            self.mask_paths = [self.mask_paths[i] for i in indices]
            self.labels = [self.labels[i] for i in indices]

    def __data_generation(self, batch_image_paths, batch_mask_paths, batch_labels):
        images = np.empty((self.batch_size, *self.input_size, 1))
        masks = np.empty((self.batch_size, *self.input_size, 1))
        labels = np.empty((self.batch_size), dtype=int)

        for i, (image_path, mask_path, label) in enumerate(zip(batch_image_paths, batch_mask_paths, batch_labels)):
            image = load_img(image_path, color_mode='grayscale', target_size=self.input_size)
            mask = load_img(mask_path, color_mode='grayscale', target_size=self.input_size)

            image = img_to_array(image) / 255.0
            mask = img_to_array(mask) / 255.0

            images[i,] = image
            masks[i,] = mask
            labels[i] = label

        return images, masks, labels

# Baseline Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Multiply, Add, Activation
from tensorflow.keras.models import Model

def attention_module(inputs):
    avg_pool = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    max_pool = tf.keras.layers.GlobalMaxPooling2D()(inputs)

    avg_dense = Dense(inputs.shape[-1] // 8, activation='relu')(avg_pool)
    avg_dense = Dense(inputs.shape[-1], activation='sigmoid')(avg_dense)

    max_dense = Dense(inputs.shape[-1] // 8, activation='relu')(max_pool)
    max_dense = Dense(inputs.shape[-1], activation='sigmoid')(max_dense)

    channel_attention = Add()([avg_dense, max_dense])
    channel_attention = Activation('sigmoid')(channel_attention)

    channel_attention = Multiply()([inputs, channel_attention])
    return channel_attention

def create_cnn_with_attention(input_shape=(256, 256, 1), num_classes=4):
    im_input = Input(shape=input_shape)
    mask_input = Input(shape=input_shape)
    classification_input = tf.keras.layers.Multiply()([im_input, mask_input])

    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(classification_input)
    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = attention_module(x)

    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = attention_module(x)

    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = attention_module(x)

    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    class_output = Dense(num_classes, activation='softmax', name='class_output')(x)

    model = Model(inputs=[im_input, mask_input], outputs=class_output)
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
    return model

model = create_cnn_with_attention()
model.summary()


Model: "model_1502"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3005 (InputLayer)         [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
input_3006 (InputLayer)         [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
multiply_6008 (Multiply)        (None, 256, 256, 1)  0           input_3005[0][0]                 
                                                                 input_3006[0][0]                 
__________________________________________________________________________________________________
conv2d_9012 (Conv2D)            (None, 254, 254, 32) 320         multiply_6008[0][0]     

### Client logic

In [ ]:
def client_update(model, data_generator, epochs=1, byzantine=False, attack_type=None):
    if not byzantine:
        model.fit(data_generator, epochs=epochs, verbose=1)
        return model.get_weights()
    
    if attack_type == 'random_update':
        return [np.random.randn(*w.shape) for w in model.get_weights()]
    
    elif attack_type == 'scaling_attack':
        scale_factor = 100
        model.fit(data_generator, epochs=epochs, verbose=0)
        return [w * scale_factor for w in model.get_weights()]
    
    elif attack_type == 'sign_flipping':
        model.fit(data_generator, epochs=epochs, verbose=0)
        return [-w for w in model.get_weights()]
    
    elif attack_type == 'targeted_attack':
        model.fit(data_generator, epochs=epochs, verbose=0)
        targeted_weights = model.get_weights()
        for layer in targeted_weights:
            layer += np.random.normal(0, 1, size=layer.shape)
        return targeted_weights

### MultiKrum logic

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
def multi_krum(updates, num_to_select, f):
    distances = euclidean_distances([np.concatenate([u.flatten() for u in update]) for update in updates])

    scores = []
    for i in range(len(updates)):
        sorted_distances = np.sort(distances[i])
        scores.append(np.sum(sorted_distances[:len(updates) - f - 1]))
    
    selected_indices = np.argsort(scores)[:num_to_select]
    print(f"Selected clients: {selected_indices}")
    return selected_indices

def apply_weights(model, weights):
    model.set_weights(weights)


### FoolsGold logic

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1 * norm2)

def compute_similarity(grad1, grad2):
    dot_product = np.dot(grad1.flatten(), grad2.flatten())
    norm1 = np.linalg.norm(grad1)
    norm2 = np.linalg.norm(grad2)
    return dot_product / (norm1 * norm2)

def fools_gold_weighting(client_gradients):
    num_clients = len(client_gradients)
    sim_matrix = np.zeros((num_clients, num_clients))

    for i in range(num_clients):
        for j in range(i + 1, num_clients):
            sim = cosine_similarity(client_gradients[i][0], client_gradients[j][0])
            sim_matrix[i, j] = sim_matrix[j, i] = sim

    client_weights = np.ones(num_clients)

    for i in range(num_clients):
        max_sim = np.max(sim_matrix[i, :])
        client_weights[i] = 1 - max_sim

    client_weights = np.clip(client_weights, 0, 1)
    print(f"Weights assigned to the chosen clients: {client_weights}")
    return client_weights

def apply_fools_gold_weights(updates, client_weights):
    weighted_updates = [ [layer * client_weights[i] for layer in updates[i]] for i in range(len(updates)) ]
    return np.sum(weighted_updates, axis=0) / np.sum(client_weights)

# Data prep

In [ ]:
DATA_PATH = "path_to_dataset_dir/COVID-19_Radiography_Dataset"
label_map = {
'COVID': 0,
'Lung_Opacity': 1,
'Normal': 2,
'Viral Pneumonia': 3
}
image_paths = []
mask_paths = []
labels = []
for label_name, label_index in label_map.items():
    images_dir = os.path.join(DATA_PATH, label_name, 'images')
    masks_dir = os.path.join(DATA_PATH, label_name, 'masks')
    image_files = os.listdir(images_dir)

    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)

        image_paths.append(image_path)
        mask_paths.append(mask_path)
        labels.append(label_index)

for label_name, label_index in label_map.items():
    images_dir = os.path.join(DATA_PATH, label_name, 'images')
    masks_dir = os.path.join(DATA_PATH, label_name, 'masks')
    image_files = os.listdir(images_dir)
    
    for image_file in image_files:
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)

        image_paths.append(image_path)
        mask_paths.append(mask_path)
        labels.append(label_index)

image_paths_train, image_paths_test, mask_paths_train, mask_paths_test, labels_train, labels_test = train_test_split(
    image_paths, mask_paths, labels, test_size=0.1, random_state=42)

In [ ]:
import math
from scipy.stats import binom
def determine_k(N, f, delta=0.75, tolerance=0.05):
    expected_malicious = f / N

    for k in range(math.ceil(N / 2), N + 1):
        expected_malicious_in_k = k * expected_malicious
        
        probability_bound = np.exp(-delta**2 * expected_malicious_in_k / 3)
        
        if probability_bound <= tolerance:
            return k
    
    return N

In [19]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

# FL training scenarios

In [ ]:
def federated_simulation_with_fools_gold(image_paths, mask_paths, labels, num_clients=5, num_rounds=10, f=1, attack_type='random_update'):
    model = create_cnn_with_attention()

    image_splits = np.array_split(image_paths, num_clients)
    mask_splits = np.array_split(mask_paths, num_clients)
    labels_splits = np.array_split(labels, num_clients)

    local_generators = []
    for client_id in range(num_clients):
        generator = DataGenerator(image_splits[client_id], mask_splits[client_id], labels_splits[client_id], batch_size=32, is_segmentation=True)
        local_generators.append(generator)

    byzantine_clients = random.sample(range(num_clients), f)

    delta = 0.75
    tolerance = 0.1
    f_est = num_clients//3
    m = determine_k(num_clients, f_est, delta, tolerance)
    print(f"NUMER OF SELECTED CLIENTS: {m}")

    global_model_performance = []
    byzantine_detected_rounds = []
    all_client_gradients = [[] for _ in range(num_clients)]

    for round_num in range(num_rounds):
        print(f"Round {round_num+1}/{num_rounds}")

        updates = []
        client_gradients = []
        for client_id, generator in enumerate(local_generators):
            local_model = create_cnn_with_attention()
            local_model.set_weights(model.get_weights())

            byzantine = client_id in byzantine_clients
            if byzantine:
                print(f"Client {client_id} is Byzantine")
            client_weights = client_update(local_model, generator, byzantine=byzantine, epochs=1, attack_type=attack_type)
            updates.append(client_weights)

            flattened_weights = np.concatenate([layer.flatten() for layer in client_weights])
            client_gradients.append(flattened_weights)

        selected_updates = multi_krum(updates, num_to_select=m, f=f)

        for i, grad in enumerate(client_gradients):
            all_client_gradients[i].append(grad)

        fg_weights = fools_gold_weighting([all_client_gradients[i] for i in selected_updates])
        aggregated_weights = apply_fools_gold_weights([updates[i] for i in selected_updates], fg_weights)
        apply_weights(model, aggregated_weights)
        print("Global model eval:")
        model.evaluate(DataGenerator(image_paths_test, mask_paths_test, labels_test, batch_size=32, is_segmentation=True, shuffle=False))
    return model, global_model_performance, byzantine_detected_rounds

num_clients = 60
num_rounds = 25
f = 20
attack_type = 'targeted_attack'

global_model, global_model_performance, byzantine_detected_rounds = federated_simulation_with_fools_gold(image_paths_train, mask_paths_train, labels_train, 
                                                                                     num_clients=num_clients, num_rounds=num_rounds, 
                                                                                     f=f, attack_type=attack_type)


NUMER OF SELECTED CLIENTS: 37
Round 1/25
19/19 [==============================] - 2s 86ms/step - loss: 1.3944 - accuracy: 0.4194
Client 2 is Byzantine
19/19 [==============================] - 2s 87ms/step - loss: 1.3201 - accuracy: 0.4408
Client 5 is Byzantine
19/19 [==============================] - 2s 88ms/step - loss: 1.1883 - accuracy: 0.4901
Client 8 is Byzantine
19/19 [==============================] - 2s 87ms/step - loss: 1.2680 - accuracy: 0.4523
Client 11 is Byzantine
19/19 [==============================] - 2s 88ms/step - loss: 1.2319 - accuracy: 0.4868
Client 15 is Byzantine
Client 16 is Byzantine
19/19 [==============================] - 3s 91ms/step - loss: 1.2447 - accuracy: 0.4770
Client 23 is Byzantine
Client 24 is Byzantine
19/19 [==============================] - 2s 89ms/step - loss: 1.2526 - accuracy: 0.4359
Client 26 is Byzantine
19/19 [==============================] - 3s 90ms/step - loss: 1.3358 - accuracy: 0.4523
Client 30 is Byzantine
19/19 [=====================